In [157]:
import os
import yaml
import time
from tqdm import tqdm
from src.utils import batch_generate, tokens_generate, run_inference
from src.mem import check_memory
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

with open("config/config.yaml", "r") as f: 
    config = yaml.safe_load(f)

model_str = 'deepseek_r1_qwendistill_1.5'
eval_df = 'big_bench'

ds = config['datasets'][eval_df]
model_path = config['models'][model_str]['path']

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [139]:
dataset = load_dataset(ds['source'], ds['subset']).shuffle(config['seed'])
tokenizer = AutoTokenizer.from_pretrained(model_path) 
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

# Experiment
Goal: test model performance on 100 BBH questions w/ chain of thought reasoning =D

In [178]:
df = pd.DataFrame(dataset['train'][0:10])
df['input'] = apply_instruct_format(df['input'], model = model_str, is_math = True)

batches = batch_generate(df, ds['input_column'], ds['target_column'])
tokens = tokens_generate(batches, tokenizer, device = 'mps')
res = run_inference(model, tokens, tokenizer, time_tracking = True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [221]:
for result in res: 
    print(result)

{'batch_idx': 0, 'input': {'input_ids': tensor([[151643, 151643, 151643,  ...,  77091, 151648,    198],
        [151646,     27,    318,  ...,  77091, 151648,    198],
        [151643, 151643, 151643,  ...,  77091, 151648,    198],
        [151643, 151643, 151643,  ...,  77091, 151648,    198],
        [151643, 151643, 151643,  ...,  77091, 151648,    198]],
       device='mps:0'), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='mps:0')}, 'response': ["<im_start|>system\nYou are an AI assistant specialized in logical and causal reasoning.<|im_end|>\n<|im_start|>user\nplease reason step by step, and put your final answer within \\boxed{}. How would a typical person answer each of the following questions about causation?\nJanet is an employee in a factory. Since she works in the maintenance department, she knows how to grease and oil all of 

In [218]:
import re
for result in res: 
    for single_response in result['response']:
        cleaned = re.sub(r'^.*?<think>', '<think>', single_response, flags=re.DOTALL)
        print(cleaned)

<think>
Okay, so I'm trying to figure out whether Kate didn't put oil in the machine causing it
<think>
Okay, so I need to figure out whether Alex caused the plants to dry out. Let me break
<think>
Okay, so I'm trying to figure out whether the motorboat started because Ned changed the motor's
<think>
Okay, so I'm trying to figure out whether the drunk driver caused the injury to Joe's son
<think>
Okay, so I need to figure out if Frank T. was intentionally shooting his neighbor in the body
<think>
Okay, so I'm trying to figure out whether Brown intentionally rolled a six on a die to deton
<think>
Okay, so I need to figure out if Susan intentionally increased the prominence of the Atlantic division. Let
<think>
Okay, so I'm trying to figure out whether the department budget committee causes the approval of Prof.
<think>
Okay, so I'm trying to figure out whether Louie won the $100 bet because
<think>
Okay, so I'm trying to figure out how a typical person would answer this question about c

In [211]:
res[0]['response'][2]

"<im_start|>system\nYou are an AI assistant specialized in logical and causal reasoning.<|im_end|>\n<|im_start|>user\nplease reason step by step, and put your final answer within \\boxed{}. How would a typical person answer each of the following questions about causation?\nNed has a new motorboat. When Ned turns the key, the motorboat starts if either the gear is in neutral or the motor is in the lock position. Today, the gear is in neutral, and the motor is not in the lock position. Ned checks the motor to see if it is in the lock position. He changes its position, and he puts it in the lock position. Because the motorboat would start if either the gear is in neutral or the motor is in the lock position, the motorboat starts when Ned turns the key. Did the motorboat start because Ned changed the position of the motor?\nOptions:\n- Yes\n- No<|im_end|>\n<|im_start|>assistant<think>\nOkay, so I'm trying to figure out whether the motorboat started because Ned changed the motor's"

In [193]:
# [print(result['response'].replace(result['input'], '').strip()) for result in res]

AttributeError: 'list' object has no attribute 'replace'

In [176]:

tokens = tokenizer(test[3], padding = True, truncation = True, return_tensors = 'pt').to(device)
response = model.generate(**tokens, temperature = 0.6, max_tokens = 2000)
full_response = tokenizer.decode(response[0], skip_special_tokens = True)

ValueError: The following `model_kwargs` are not used by the model: ['max_tokens'] (note: typos in the generate arguments will also show up in this list)

In [175]:
full_response.replace(test[3], '').strip() # nice! 

"Okay, so I'm trying to figure out whether the drunk driver caused the injury to Joe's son"